In [1]:
from sklearn.neighbors import KNeighborsClassifier
import os
os.chdir('/Users/renalkakhan/Documents/GitHub/CS598_DLH_Project/')
from dataset.preprocessing.word2vec_embeddings_gen import Word2VecFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import GloVeFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import FastTextFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import USEFeatureGeneration
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import csv
import numpy as np
import pandas as pd

In [2]:
class KNN:
    def __init__(self, x_train, y_train, x_test, y_test, n):
        self.knn = KNeighborsClassifier(n_neighbors=n)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def train(self):
        self.knn.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.knn.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        return f1_macro, f1_micro

In [3]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']
column_headings = ["Morbidity Class", "KNN1_Macro F1", "KNN1_Micro F1", "KNN5_Macro F1", "KNN5_Micro F1"]

with open("./results/word-embeddings-features/performance_KNN_W2V.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

In [4]:
all_f1_macro1_scores = []
all_f1_micro1_scores = []

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = Word2VecFeatureGeneration(train_preprocessed_df, morbidity).word2vec_matrix_gen()
    X = np.average(X, axis=1)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list1 = []
    f1_micro_list1 = []
    f1_macro_list5 = []
    f1_micro_list5 = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training KNN using TF-IDF Representation
        knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1)
        knn1_obj.train()

        f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

        f1_macro_list1.append(f1_macro1)
        f1_micro_list1.append(f1_micro1)

        knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5)
        knn5_obj.train()

        f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

        f1_macro_list5.append(f1_macro5)
        f1_micro_list5.append(f1_micro5)

    f1_macro1 = np.mean(f1_macro_list1)
    f1_micro1 = np.mean(f1_micro_list1)
    f1_macro5 = np.mean(f1_macro_list5)
    f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/word-embeddings-features/performance_KNN_W2V.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_KNN_W2V.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(572, 3570, 100) (572,) Counter({0.0: 502, 1.0: 70})
For n=1, Macro F1 score: 0.49164697401920676 and Micro F1 Score 0.7622202056866303
For n=5, Macro F1 score: 0.4808690011802418 and Micro F1 Score 0.8671506352087114
CAD
(548, 3570, 100) (548,) Counter({1.0: 325, 0.0: 223})
For n=1, Macro F1 score: 0.6456556955022531 and Micro F1 Score 0.6640740740740741
For n=5, Macro F1 score: 0.6362451356564487 and Micro F1 Score 0.6696969696969697
CHF
(243, 3122, 100) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1.0 and Micro F1 Score 1.0
For n=5, Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 3570, 100) (582,) Counter({0.0: 460, 1.0: 122})
For n=1, Macro F1 score: 0.5282519992709538 and Micro F1 Score 0.6785505552308592
For n=5, Macro F1 score: 0.5571509387088595 and Micro F1 Score 0.7783167738164817
Diabetes
(567, 3570, 100) (567,) Counter({1.0: 396, 0.0: 171})
For n=1, Macro F1 score: 0.541661399503258 and Micro F1 Score 0.6226503759398496
For n=5, Macro F1 score: 0.5

In [5]:
with open("./results/word-embeddings-features/performance_KNN_Glove.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = GloVeFeatureGeneration(train_preprocessed_df, morbidity).glove_matrix_gen()
    X = np.average(X, axis=1)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list1 = []
    f1_micro_list1 = []
    f1_macro_list5 = []
    f1_micro_list5 = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training KNN using TF-IDF Representation
        knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1)
        knn1_obj.train()

        f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

        f1_macro_list1.append(f1_macro1)
        f1_micro_list1.append(f1_micro1)

        knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5)
        knn5_obj.train()

        f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

        f1_macro_list5.append(f1_macro5)
        f1_micro_list5.append(f1_micro5)

    f1_macro1 = np.mean(f1_macro_list1)
    f1_micro1 = np.mean(f1_micro_list1)
    f1_macro5 = np.mean(f1_macro_list5)
    f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/word-embeddings-features/performance_KNN_Glove.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_KNN_Glove.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma


/Users/renalkakhan/Documents/GitHub/CS598_DLH_Project/dataset/preprocessing/word2vec_embeddings_gen.py:74: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X[i, :, :] = np.array(sentence_vectors)


ValueError: setting an array element with a sequence.

In [ ]:
with open("./results/word-embeddings-features/performance_KNN_FastText.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FastTextFeatureGeneration(train_preprocessed_df, morbidity).fasttext_matrix_gen()
    X = np.average(X, axis=1)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list1 = []
    f1_micro_list1 = []
    f1_macro_list5 = []
    f1_micro_list5 = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training KNN using TF-IDF Representation
        knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1)
        knn1_obj.train()

        f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

        f1_macro_list1.append(f1_macro1)
        f1_micro_list1.append(f1_micro1)

        knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5)
        knn5_obj.train()

        f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

        f1_macro_list5.append(f1_macro5)
        f1_micro_list5.append(f1_micro5)

    f1_macro1 = np.mean(f1_macro_list1)
    f1_micro1 = np.mean(f1_micro_list1)
    f1_macro5 = np.mean(f1_macro_list5)
    f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/word-embeddings-features/performance_KNN_FastText.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_KNN_FastText.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(572, 3570, 100) (572,) Counter({0.0: 502, 1.0: 70})
For n=1, Macro F1 score: 0.47344501352755175 and Micro F1 Score 0.7533877797943134
For n=5, Macro F1 score: 0.46799918748723146 and Micro F1 Score 0.8513914095583788
CAD
(548, 3570, 100) (548,) Counter({1.0: 325, 0.0: 223})
For n=1, Macro F1 score: 0.600666529084805 and Micro F1 Score 0.625959595959596
For n=5, Macro F1 score: 0.6084171607469139 and Micro F1 Score 0.6385521885521885
CHF
(243, 3122, 100) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1.0 and Micro F1 Score 1.0
For n=5, Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 3570, 100) (582,) Counter({0.0: 460, 1.0: 122})
For n=1, Macro F1 score: 0.5341621253751527 and Micro F1 Score 0.7077440093512566
For n=5, Macro F1 score: 0.5405894612255174 and Micro F1 Score 0.7748684979544127
Diabetes
(567, 3570, 100) (567,) Counter({1.0: 396, 0.0: 171})
For n=1, Macro F1 score: 0.5511488017164803 and Micro F1 Score 0.6348997493734335
For n=5, Macro F1 score: 0.5

In [ ]:
with open("./results/word-embeddings-features/performance_KNN_USE.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = USEFeatureGeneration(train_preprocessed_df, morbidity).use_matrix_gen()
    X = np.average(X, axis=1)
    X = X.reshape(-1, 1)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list1 = []
    f1_micro_list1 = []
    f1_macro_list5 = []
    f1_micro_list5 = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training KNN using TF-IDF Representation
        knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1)
        knn1_obj.train()

        f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

        f1_macro_list1.append(f1_macro1)
        f1_micro_list1.append(f1_micro1)

        knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5)
        knn5_obj.train()

        f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

        f1_macro_list5.append(f1_macro5)
        f1_micro_list5.append(f1_micro5)

    f1_macro1 = np.mean(f1_macro_list1)
    f1_micro1 = np.mean(f1_micro_list1)
    f1_macro5 = np.mean(f1_macro_list5)
    f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/word-embeddings-features/performance_KNN_USE.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_KNN_USE.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma


2023-04-30 20:16:17.566371: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(572, 512) (572,) Counter({0.0: 502, 1.0: 70})
For n=1, Macro F1 score: 0.47964358364388515 and Micro F1 Score 0.7725952813067151
For n=5, Macro F1 score: 0.46474568561995094 and Micro F1 Score 0.8688445251058681
CAD
(548, 512) (548,) Counter({1.0: 325, 0.0: 223})
For n=1, Macro F1 score: 0.5549841575818227 and Micro F1 Score 0.5765656565656566
For n=5, Macro F1 score: 0.6128304653226418 and Micro F1 Score 0.6404040404040404
CHF
(243, 512) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1.0 and Micro F1 Score 1.0
For n=5, Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 512) (582,) Counter({0.0: 460, 1.0: 122})
For n=1, Macro F1 score: 0.48368929474702577 and Micro F1 Score 0.6700467562828755
For n=5, Macro F1 score: 0.4770184414617943 and Micro F1 Score 0.7576563413208651
Diabetes
(567, 512) (567,) Counter({1.0: 396, 0.0: 171})
For n=1, Macro F1 score: 0.5620143684813991 and Micro F1 Score 0.6386904761904761
For n=5, Macro F1 score: 0.6186664937582427 and Micro F1 Score